In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **1. Chuẩn bị môi trường**

In [ ]:
!pip -q install ultralytics opencv-python pillow imagehash

from ultralytics import YOLO
from pathlib import Path
import os, random, cv2, numpy as np,torch
from collections import Counter, defaultdict
import collections
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd, shutil, math, json
from datetime import datetime
import yaml

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ["PYTHONHASHSEED"] = str(SEED)


## **2. Khai báo đường dẫn & tạo data.yaml**

In [ ]:
root = Path("/kaggle/input/traffic-vehicles-object-detection/Traffic Dataset")

data_yaml = f"""
path: "{root}"
train: images/train
val: images/val
test: images/test
names: [Car, Number Plate, Blur Number Plate, Two Wheeler, Auto, Bus, Truck]
"""
Path("/kaggle/working/data.yaml").write_text(data_yaml)
print(Path("/kaggle/working/data.yaml").read_text())

## **3. EDA và Sanity check**

#### 3.1 Đếm ảnh/nhãn từng split

In [ ]:
def count_images(p):
    return sum(1 for f in Path(p).glob("*") if f.suffix.lower() in {".jpg",".jpeg",".png"})

def count_labels(p):
    return sum(1 for f in Path(p).glob("*.txt"))

for split in ["train","val","test"]:
    n_i = count_images(root/f"images/{split}")
    n_l = count_labels(root/f"labels/{split}") if split!="test" else "-"
    print(f"{split}: images={n_i}, labels={n_l}")

#### 3.2 Phân bố lớp (để biết lớp nào hiếm)

In [ ]:
cls_names = ["Car","Number Plate","Blur Number Plate","Two Wheeler","Auto","Bus","Truck"]
cnt = Counter()
for split in ["train","val"]:
    for txt in (root/f"labels/{split}").glob("*.txt"):
        for line in txt.read_text().strip().splitlines():
            if not line: continue
            cid = int(line.split()[0])
            cnt[cid]+=1

print("Class distribution (train+val):")
for i,n in sorted(cnt.items()):
    print(f"{i}:{cls_names[i]} -> {n}")


#### 3.3 Kích thước ảnh (mẫu) & hiển thị vài ảnh có bbox

##### 3.3.1 Lọc ảnh hợp lệ theo đuôi file (suffix)

In [ ]:
# 3.3.1 – Collect valid image paths (train split)
img_dir = root / "images" / "train"

imgs = []
for ext in ("*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG"):
    imgs.extend(list(img_dir.glob(ext)))

print(f"Total valid images found (train): {len(imgs)}")
print("Sample 5:", [p.name for p in imgs[:5]])


##### 3.3.2 Thống kê kích thước ảnh & vẽ histogram cạnh ngắn

In [ ]:
# 3.3.2 – Size stats + histogram of shorter edge
size_ctr = collections.Counter()
short_edges = []

for p in imgs:
    im = cv2.imread(str(p))
    if im is None:
        continue
    h, w = im.shape[:2]
    size_ctr[(w, h)] += 1
    short_edges.append(min(w, h))

print("Top 8 most common sizes:")
for (wh, cnt) in size_ctr.most_common(8):
    print(f"{wh} -> {cnt}")

plt.figure(figsize=(6,3.5))
plt.hist(short_edges, bins=20)
plt.title("Shorter edge distribution (train)")
plt.xlabel("Pixels")
plt.ylabel("Count")
plt.show()


##### 3.3.3 Kiểm tra lỗi nhãn (format, range, zero-area)

In [ ]:
# 3.3.3 – Label sanity checks 
label_dir = root / "labels" / "train"

issues = {"format": 0, "range": 0, "zero_area": 0}
total_boxes = 0
checked_files = 0
bad_rows = []  # lưu chi tiết file và dòng lỗi

for txt in label_dir.glob("*.txt"):
    checked_files += 1
    content = txt.read_text().strip().splitlines()
    for line_no, line in enumerate(content, 1):
        if not line.strip():
            continue
        parts = line.split()
        # expected: c x y w h (all normalized in [0,1])
        if len(parts) != 5 or not parts[0].isdigit():
            issues["format"] += 1
            bad_rows.append((txt.name, line_no, "format", line))
            continue
        c, x, y, bw, bh = parts
        try:
            x = float(x); y = float(y); bw = float(bw); bh = float(bh)
        except:
            issues["format"] += 1
            bad_rows.append((txt.name, line_no, "format", line))
            continue
        total_boxes += 1

        # Kiểm tra phạm vi [0,1]
        if not (0 <= x <= 1 and 0 <= y <= 1 and 0 < bw <= 1 and 0 < bh <= 1):
            issues["range"] += 1
            bad_rows.append((txt.name, line_no, "range", line))

        # Kiểm tra diện tích > 0
        if (bw * bh) <= 1e-6:
            issues["zero_area"] += 1
            bad_rows.append((txt.name, line_no, "zero_area", line))

print(f"Checked files: {checked_files} | Checked boxes: {total_boxes}")
print("Issues summary:", issues)

# Hiển thị chi tiết các lỗi (nếu có)
if bad_rows:
    print("\n Detailed error list (first 10 shown):")
    for f, ln, reason, line in bad_rows[:10]:
        print(f"{f} line {ln} | reason: {reason} | {line}")
    print(f"\nTotal bad rows: {len(bad_rows)}")
else:
    print("\n No label errors detected.")


Fix label errors automatically (if found)

In [ ]:
# ==== INPUT ROOT ====
IN_ROOT = root  # Path("/kaggle/input/traffic-vehicles-object-detection/Traffic Dataset")

# ==== OUTPUT ROOT ====
OUT_ROOT = Path("/kaggle/working/traffic_dataset_clean")

# (tùy chọn) xóa sạch thư mục OUT_ROOT nếu bạn muốn ghi đè mỗi lần chạy
def rm_tree(p: Path):
    if p.exists():
        for c in p.iterdir():
            if c.is_dir():
                rm_tree(c)
            else:
                c.unlink(missing_ok=True)
        p.rmdir()

# rm_tree(OUT_ROOT)  # bỏ comment nếu muốn xóa sạch trước

# đảm bảo cấu trúc thư mục
for d in [
    "images/train","images/val","images/test",
    "labels/train","labels/val"
]:
    (OUT_ROOT/d).mkdir(parents=True, exist_ok=True)

# ---- helper: tạo link/copy an toàn, không lỗi khi file tồn tại ----
def link_or_copy(src: Path, dst: Path, overwrite: bool = False):
    # nếu đích đã tồn tại
    if dst.exists():
        try:
            # nếu là cùng file thì bỏ qua
            if os.path.samefile(src, dst):
                return
        except Exception:
            pass
        if overwrite:
            try:
                dst.unlink()
            except IsADirectoryError:
                shutil.rmtree(dst)
        else:
            return  # không ghi đè -> bỏ qua

    # ưu tiên symlink (nhanh/nhẹ), nếu lỗi thì fallback copy
    try:
        os.symlink(src, dst)
    except Exception:
        shutil.copy2(src, dst)

# ==== 1) Copy/Symlink IMAGES sang OUT_ROOT ====
valid_ext = {".jpg",".jpeg",".png",".JPG",".JPEG",".PNG"}

for split in ["train","val","test"]:
    in_dir = IN_ROOT / f"images/{split}"
    out_dir = OUT_ROOT / f"images/{split}"
    for img in in_dir.glob("*"):
        if img.suffix.lower() not in valid_ext:
            continue
        link_or_copy(img, out_dir / img.name, overwrite=False)

# ==== 2) Gộp các dòng lỗi theo file để xoá (sử dụng bad_rows có sẵn) ====
# CHÚ Ý: dùng .name để key là string, khớp với txt.name ở bước 3
to_delete = defaultdict(set)  # {file_name(str): {line_numbers}}
for fname, line_no, reason, line in bad_rows:
    # fname có thể là Path -> lấy tên file thuần
    file_key = fname.name if hasattr(fname, "name") else str(fname)
    to_delete[file_key].add(line_no)

# ==== 3) Ghi LABELS sạch cho TRAIN ====
fixed_count = 0
for txt in (IN_ROOT/"labels/train").glob("*.txt"):
    lines = txt.read_text().splitlines()
    rm = to_delete.get(txt.name, set())
    if rm:
        new_lines = [ln for i, ln in enumerate(lines, 1) if i not in rm]
        fixed_count += len(rm)
    else:
        new_lines = lines
    (OUT_ROOT/"labels/train"/txt.name).write_text("\n".join(new_lines))

# ==== 4) VAL giữ nguyên ====
for txt in (IN_ROOT/"labels/val").glob("*.txt"):
    link_or_copy(txt, OUT_ROOT/"labels/val"/txt.name, overwrite=False)

print(f"✅ Cleaned {fixed_count} bad label lines. Clean dataset at: {OUT_ROOT}")

# ==== 5) Viết data.yaml trỏ tới OUT_ROOT ====
data_yaml = f"""
path: "{OUT_ROOT}"
train: images/train
val: images/val
test: images/test
names: [Car, Number Plate, Blur Number Plate, Two Wheeler, Auto, Bus, Truck]
"""
Path("/kaggle/working/data.yaml").write_text(data_yaml)
print(Path("/kaggle/working/data.yaml").read_text())

# ==== 6) Re-check nhanh trên OUT_ROOT ====
issues = {"format":0,"range":0,"zero_area":0}
for txt in (OUT_ROOT/"labels/train").glob("*.txt"):
    for line in txt.read_text().splitlines():
        if not line.strip(): 
            continue
        parts = line.split()
        if len(parts)!=5 or not parts[0].isdigit():
            issues["format"]+=1; continue
        _,x,y,bw,bh = parts
        x=float(x); y=float(y); bw=float(bw); bh=float(bh)
        if not (0<=x<=1 and 0<=y<=1 and 0<bw<=1 and 0<bh<=1): 
            issues["range"]+=1
        if bw*bh <= 1e-6: 
            issues["zero_area"]+=1
print("Recheck on CLEAN labels:", issues)


##### 3.3.4 Hiển thị 2–3 ảnh mẫu với bounding boxes

In [ ]:
# 3.3.4 – Visualize a few images with labels
# Từ label -> path ảnh (tự tìm đúng ext)
def img_path_from_label(lfile: Path) -> Path | None:
    stem = lfile.stem
    img_dir = OUT_ROOT / "images" / "train"
    for ext in [".jpg",".jpeg",".png",".JPG",".JPEG",".PNG"]:
        p = img_dir / f"{stem}{ext}"
        if p.exists(): return p
    return None

# Lấy danh sách label có ít nhất 1 bbox
label_files = []
for txt in sorted((OUT_ROOT/"labels/train").glob("*.txt")):
    if any(line.strip() for line in txt.read_text().splitlines()):
        label_files.append(txt)

print("Labels with at least 1 box:", len(label_files))
def draw_img_with_boxes(ax, img_path: Path, lbl_path: Path):
    im_bgr = cv2.imread(str(img_path))
    h, w = im_bgr.shape[:2]
    im = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)
    ax.imshow(im); ax.axis('off')
    if not lbl_path.exists(): return
    for line in lbl_path.read_text().splitlines():
        if not line.strip(): continue
        c,x,y,bw,bh = line.split()
        c = int(c); x=float(x); y=float(y); bw=float(bw); bh=float(bh)
        x1 = (x-bw/2)*w; y1=(y-bh/2)*h; ww=bw*w; hh=bh*h
        rect = patches.Rectangle((int(x1),int(y1)), int(ww),int(hh), linewidth=2, edgecolor='lime', facecolor='none')
        ax.add_patch(rect)
        ax.text(int(x1), max(0,int(y1)-3), cls_names[c], color='yellow', fontsize=9, backgroundcolor='black')

# lấy 6 ảnh có nhãn
sel = random.sample(label_files, min(6, len(label_files)))

cols = 3; rows = int(np.ceil(len(sel)/cols))
fig, axes = plt.subplots(rows, cols, figsize=(cols*6, rows*4))
axes = np.array(axes).reshape(-1)

for ax, lf in zip(axes, sel):
    ip = img_path_from_label(lf)
    draw_img_with_boxes(ax, ip, lf)

# tắt ô trống
for ax in axes[len(sel):]:
    ax.axis('off')

plt.suptitle("Sample labeled images (train)", y=0.99, fontsize=14)
plt.tight_layout()
plt.show()



## **4. Baseline yolov8n**

#### 4.1 Cấu hình thí nghiệm (tên exp, tham số chính)

In [ ]:
DATA_YAML = "/kaggle/working/data.yaml"   # dùng bản CLEAN bạn vừa tạo
EXP_Y8N = "y8n_baseline_736"

CFG_Y8N = dict(
    data=DATA_YAML,
    imgsz=736,          # giữ chi tiết biển số; nếu OOM đổi 640
    epochs=60,
    batch=16,           # nếu OOM: 8 hoặc 4
    workers=2,
    seed=42,
    optimizer="AdamW",
    lr0=0.01,
    weight_decay=8e-4,
    warmup_epochs=3.0,
    patience=20,        # early stop
    # augment hợp lý cho dataset xe cộ, hỗ trợ lớp hiếm
    mosaic=1.0, mixup=0.2, copy_paste=0.1,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    translate=0.15, scale=0.5, shear=0.2, degrees=10,
    fliplr=0.5, flipud=0.1,
    project="/kaggle/working", name=EXP_Y8N
)
CFG_Y8N

#### 4.2 Huấn luyện baseline

In [ ]:
model = YOLO("yolov8n.pt")
results_Y8N = model.train(**CFG_Y8N)
print(f"✅ Trained: {EXP_Y8N}")


## **5. yolov8s**

In [ ]:
# 5.1 Fine-tuning YOLOv8s (higher capacity + higher resolution)
EXP_Y8S = "y8s_896"

model = YOLO("yolov8s.pt")  # pre-trained base model

CFG_Y8S = {
    "data": DATA_YAML,
    "epochs": 100,
    "imgsz": 896,
    "batch": 16,
    "optimizer": "AdamW",
    "lr0": 0.001,
    "patience": 20,
    "augment": True,
    "hsv_h": 0.015,
    "hsv_v": 0.5,
    "mixup": 0.2,
    "mosaic": 1.0,
    "project": "/kaggle/working",
    "name": EXP_Y8S,
    "device": 0
}

results_Y8S = model.train(**CFG_Y8S)


## **6. yolov9m**

In [ ]:
model = YOLO("yolov9m.pt")
EXP_Y9M = "y9m_896"
model.train(
    data=DATA_YAML,
    epochs=100,
    imgsz=896,          # tăng size → bắt xe nhỏ tốt hơn
    batch=8,

    # ---------- Optimizer / LR ----------
    optimizer="AdamW",
    lr0=0.01,           # giảm nhẹ LR để ổn định
    lrf=0.05,
    momentum=0.94,
    weight_decay=0.0012,  # tăng regularize → tránh overfit

    # ---------- Warmup ----------
    warmup_epochs=4.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,

    # ---------- Strong Augment ----------
    hsv_h=0.02,
    hsv_s=0.8,
    hsv_v=0.5,
    translate=0.2,
    scale=0.65,
    shear=0.25,
    flipud=0.25,
    fliplr=0.6,
    mixup=0.3,
    mosaic=1.0,
    copy_paste=0.2,
    degrees=20,

    # ---------- Loss weighting ----------
    box=8.0,
    cls=1.0,
    dfl=1.5,
    label_smoothing=0.0,

    # ---------- Early stop ----------
    patience=30,

    # ---------- Others ----------
    project= "/kaggle/working",
    name= EXP_Y9M,
    exist_ok=True,
    verbose=True,
    seed=42
)

## **7. LẬP BẢNG SO SÁNH KẾT QUẢ CÁC MODEL TRÊN TẬP VAL**

In [ ]:
# Khai báo các exp + đường dẫn weights + imgsz (điền đúng tên exp đã train)
EXP_Y8N = "y8n_baseline_736"           
EXP_Y8S = "y8s_896"   
EXP_Y9M = "y9m_896"                       

CANDIDATES = [
    {"exp": EXP_Y8N, "weights": f"/kaggle/working/{EXP_Y8N}/weights/best.pt", "imgsz": 736},
    {"exp": EXP_Y8S, "weights": f"/kaggle/working/{EXP_Y8S}/weights/best.pt", "imgsz": 896},
    {"exp": EXP_Y9M, "weights": f"/kaggle/working/{EXP_Y9M}/weights/best.pt", "imgsz": 896},
]

def _get(rd, *keys):
    """Lấy value an toàn cho các key metric khác nhau giữa version Ultralytics."""
    for k in keys:
        if k in rd and rd[k] is not None:
            return rd[k]
    return float("nan")

def eval_on_val(weights_path: str, imgsz: int, exp_name: str, data_yaml: str):
    """Val trên split=val, trả về dict metric chính."""
    if not os.path.exists(weights_path):
        return {
            "exp": exp_name, "weights": weights_path, "imgsz": imgsz,
            "mAP50": float("nan"), "mAP50-95": float("nan"),
            "precision": float("nan"), "recall": float("nan"),
            "note": "weights not found"
        }
    m = YOLO(weights_path)
    met = m.val(data=data_yaml, split="val", imgsz=imgsz, conf=0.001, plots=False, save_json=False)
    rd = getattr(met, "results_dict", {}) or {}
    return {
        "exp": exp_name,
        "weights": weights_path,
        "imgsz": imgsz,
        "mAP50":    _get(rd, "metrics/mAP50(B)", "metrics/mAP50"),
        "mAP50-95": _get(rd, "metrics/mAP50-95(B)", "metrics/mAP50-95"),
        "precision":_get(rd, "metrics/precision(B)", "metrics/precision"),
        "recall":   _get(rd, "metrics/recall(B)", "metrics/recall"),
        "note": ""
    }

rows = [eval_on_val(c["weights"], c["imgsz"], c["exp"], DATA_YAML) for c in CANDIDATES]
df_cmp = pd.DataFrame(rows)

# Sắp xếp theo mAP50-95 giảm dần (NaN sẽ bị đẩy xuống)
df_cmp["mAP50-95_filled"] = df_cmp["mAP50-95"].fillna(-1.0)
df_cmp = df_cmp.sort_values("mAP50-95_filled", ascending=False).drop(columns=["mAP50-95_filled"])

# Lưu bảng so sánh
cmp_csv = "/kaggle/working/val_compare_models.csv"
df_cmp.to_csv(cmp_csv, index=False)
print("===== VAL COMPARISON (sorted by mAP50-95) =====")
print(df_cmp.to_string(index=False))
print("Saved:", cmp_csv)

# Chọn best theo mAP50-95 (nếu toàn NaN thì raise)
if df_cmp["mAP50-95"].notna().any():
    best_row = df_cmp.iloc[0]
else:
    raise RuntimeError("Không có metric mAP50-95 hợp lệ cho bất kỳ model nào. Kiểm tra lại quá trình train/val.")

BEST_WEIGHTS = best_row["weights"]
BEST_IMGSZ   = int(best_row["imgsz"])
BEST_EXP     = str(best_row["exp"])
BEST_ALIAS   = "/kaggle/working/best_overall.pt"

# Copy best về alias cố định để bước 8 dùng
shutil.copy2(BEST_WEIGHTS, BEST_ALIAS)

# Ghi file JSON để bước 8 đọc lại (tránh lệ thuộc biến phiên làm việc)
best_meta = {"exp": BEST_EXP, "weights": BEST_ALIAS, "imgsz": BEST_IMGSZ}
with open("/kaggle/working/best_selection.json", "w") as f:
    json.dump(best_meta, f, indent=2)

print("\n🏆 BEST MODEL SELECTED")
print(f"  Exp   : {BEST_EXP}")
print(f"  Weights: {BEST_ALIAS}")
print(f"  imgsz : {BEST_IMGSZ}")
# =================================================================================================


## **8. Dự đoán test DÙNG BEST MODEL**

#### 8.1) Dùng best model để predict trên test và lưu kết quả (ảnh đã có bbox)

In [ ]:
# Đường dẫn metadata best (đã tạo ở bước 7)
BEST_META_JSON = "/kaggle/working/best_selection.json"
with open(BEST_META_JSON, "r") as f:
    best_meta = json.load(f)

BEST_ALIAS = best_meta["weights"]      # "/kaggle/working/best_overall.pt"
BEST_IMGSZ = int(best_meta["imgsz"])   # imgsz phù hợp với best

# Lấy test_dir từ DATA_YAML
with open(DATA_YAML, "r") as f:
    y = yaml.safe_load(f)
base = Path(y.get("path", "."))
test_entry = y.get("test", "images/test")
test_dir = Path(test_entry) if Path(test_entry).is_absolute() else (base / test_entry)
assert test_dir.exists(), f"Test dir not found: {test_dir}"

# Predict & lưu ảnh có bbox
best_model = YOLO(BEST_ALIAS)
PRED_NAME = "pred_best_test"
PRED_DIR  = f"/kaggle/working/{PRED_NAME}"

best_model.predict(
    source=test_dir,
    imgsz=BEST_IMGSZ,
    conf=0.25, iou=0.6,
    save=True, save_txt=True, save_conf=True,   # -> ảnh có bbox sẽ được vẽ và lưu
    project="/kaggle/working",
    name=PRED_NAME
)

print(f"[OK] Saved predictions with boxes to: {PRED_DIR}")


#### 8.2) Sao chép ảnh test đã vẽ bbox sang 1 folder để có thể check

In [ ]:
src_dir = Path(PRED_DIR)              
dst_dir = Path("/kaggle/working/test_with_boxes")
dst_dir.mkdir(parents=True, exist_ok=True)

# Chỉ copy ảnh (jpg/png/jpeg) đã được vẽ bbox
img_exts = (".jpg", ".jpeg", ".png")
num_copied = 0
for p in list(src_dir.glob("*.jpg")) + list(src_dir.glob("*.jpeg")) + list(src_dir.glob("*.png")):
    shutil.copy2(p, dst_dir / p.name)
    num_copied += 1

print(f"[OK] Copied {num_copied} images with boxes to: {dst_dir}")


#### 8.3) In ra một vài ảnh test đã lưu sau khi dự đoán

In [ ]:
show_dir = Path("/kaggle/working/test_with_boxes")   
imgs = [*show_dir.glob("*.jpg"), *show_dir.glob("*.jpeg"), *show_dir.glob("*.png")]
print(f"[INFO] Found {len(imgs)} images in {show_dir}")

sample_n = min(6, len(imgs))
for p in random.sample(imgs, sample_n):
    im = cv2.cvtColor(cv2.imread(str(p)), cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6,4))
    plt.imshow(im); plt.axis("off"); plt.title(p.name)
    plt.show()
